In [142]:
import pandas as pd
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, load_robot_execution_failures
from tsfresh.feature_extraction import extract_features, extract_features_on_sub_features
from tsfresh.feature_selection import select_features
from tsfresh.feature_extraction.settings import MinimalFCParameters

from tsfresh.feature_extraction.gen_features_dicts_function import derive_features_dictionaries

import matplotlib.pyplot as plt
from IPython.display import display
import json

# ....

In [86]:
download_robot_execution_failures()
timeseries, y = load_robot_execution_failures()
print(timeseries.head())

   id  time  F_x  F_y  F_z  T_x  T_y  T_z
0   1     0   -1   -1   63   -3   -1    0
1   1     1    0    0   62   -3   -1    0
2   1     2   -1   -1   61   -3    0    0
3   1     3   -1   -1   63   -2   -1    0
4   1     4   -1   -1   63   -3   -1    0


# Extract features from the Time Series
Let us start by demonstrating how a simple set of time series features (mean, median, max, variance, ...) are calculated from an example time series.

In [143]:
extracted_features = extract_features(timeseries, 
                                    column_id="id", 
                                    column_sort="time", 
                                    default_fc_parameters=MinimalFCParameters())
display(extracted_features.head())

Feature Extraction: 100%|██████████| 10/10 [00:04<00:00,  2.37it/s]


,F_x__sum_values,F_x__median,F_x__mean,F_x__length,F_x__standard_deviation,F_x__variance,F_x__maximum,F_x__minimum,F_y__sum_values,F_y__median,...,T_y__maximum,T_y__minimum,T_z__sum_values,T_z__median,T_z__mean,T_z__length,T_z__standard_deviation,T_z__variance,T_z__maximum,T_z__minimum
1,-14.0,-1.0,-0.933333,15.0,0.249444,0.062222,0.0,-1.0,-13.0,-1.0,...,0.0,-1.0,0.0,0.0,0.000000,15.0,0.000000,0.000000,0.0,0.0
2,-13.0,-1.0,-0.866667,15.0,0.956847,0.915556,0.0,-3.0,-10.0,-1.0,...,4.0,-5.0,-4.0,0.0,-0.266667,15.0,0.442217,0.195556,0.0,-1.0
3,-10.0,-1.0,-0.666667,15.0,0.596285,0.355556,1.0,-1.0,-8.0,0.0,...,1.0,-5.0,-4.0,0.0,-0.266667,15.0,0.442217,0.195556,0.0,-1.0
4,-6.0,0.0,-0.400000,15.0,0.952190,0.906667,1.0,-2.0,2.0,1.0,...,4.0,-6.0,-5.0,0.0,-0.333333,15.0,0.596285,0.355556,1.0,-1.0
5,-9.0,-1.0,-0.600000,15.0,0.879394,0.773333,2.0,-2.0,-4.0,0.0,...,3.0,-5.0,-2.0,0.0,-0.133333,15.0,0.618241,0.382222,1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,-1073.0,-98.0,-71.533333,15.0,36.585729,1338.515556,-25.0,-110.0,620.0,63.0,...,167.0,12.0,-232.0,-21.0,-15.466667,15.0,9.659998,93.315556,0.0,-28.0
85,143.0,8.0,9.533333,15.0,4.616877,21.315556,19.0,4.0,145.0,8.0,...,14.0,-1.0,-52.0,-2.0,-3.466667,15.0,2.156128,4.648889,0.0,-7.0
86,961.0,52.0,64.066667,15.0,38.235179,1461.928889,148.0,21.0,508.0,31.0,...,191.0,14.0,-81.0,-8.0,-5.400000,15.0,5.462600,29.840000,8.0,-10.0
87,4509.0,338.0,300.600000,15.0,57.753268,3335.440000,342.0,171.0,2122.0,154.0,...,471.0,222.0,475.0,35.0,31.666667,15.0,9.903983,98.088889,44.0,13.0


# How to extract features from an existing feature matrix
Should we find that these features themselves are not sufficiently informative for whatever reason, we can repeat the same feature extraction process using the  `extract_features_on_sub_features` function.

In principle this works as such:

1. The input time *X* series is windowed and the chosen set of N features are extracted. This returns a new matrix *M* where each column represents a particular **feature time series**.

2. For each feature in the resulting output, step 1 is repeated and for the chosen feature time series. Each new column generated can be referred to as a **sub-feature** or  **feature-dynamic**
    
3. Repeat for each column in *M*.

<img src="./notebooks/examples/features_on_features_diagram.png"> 

### Differences to `extract_features`
`extract_features_on_sub_features` shares most of the same parameters as `extract_features`

**Note:** that the resulting output of this operation can lead to an exponential number of columns generated. For instance if the input has 1 time series and we extract N features...

Below the algorithm is demonstrated on the same robot executaion failures dataset.

In [59]:
##TODO fix window size bug
extracted_sub_features = extract_features_on_sub_features(timeseries_container=timeseries,
                                    sub_feature_split=11,  # window size
                                    column_id="id",
                                    column_sort="time",
                                    sub_default_fc_parameters=MinimalFCParameters(),
                                    ##TODO: check if one of these isnt specified use the other
                                    default_fc_parameters=MinimalFCParameters())
display(extracted_sub_features.head())                                   

Feature Extraction: 100%|██████████| 10/10 [00:06<00:00,  1.43it/s]

   F_x||length__sum_values  F_x||length__median  F_x||length__mean  \
1                     15.0                  7.5                7.5   
2                     15.0                  7.5                7.5   
3                     15.0                  7.5                7.5   
4                     15.0                  7.5                7.5   
5                     15.0                  7.5                7.5   

   F_x||length__length  F_x||length__standard_deviation  \
1                  2.0                              3.5   
2                  2.0                              3.5   
3                  2.0                              3.5   
4                  2.0                              3.5   
5                  2.0                              3.5   

   F_x||length__variance  F_x||length__maximum  F_x||length__minimum  \
1                  12.25                  11.0                   4.0   
2                  12.25                  11.0                   4.0   
3       

# Interpreting the results

As can be seen, running `extract_features_on_sub_features` results in significantly more columns

## Decomposing the column names
`"F_x||length__sum_values"`

This is demonstrated below

In [45]:
## Take a subset of the columns to demonstrate (reduce size of output)
sub_feature_names = extracted_sub_features.columns.tolist()[:120]
f,ff = derive_features_dictionaries(sub_feature_names)

print("The set f features calculated on the original time series:\n")
#[print(f[k],"\n") for k in f.keys()]
print(json.dumps(f,sort_keys=True, indent=4))


The set f features calculated on the original time series:

{
    "F_x": {
        "length": null,
        "maximum": null,
        "mean": null,
        "median": null,
        "minimum": null,
        "standard_deviation": null,
        "sum_values": null,
        "variance": null
    },
    "F_y": {
        "length": null,
        "maximum": null,
        "mean": null,
        "median": null,
        "minimum": null,
        "standard_deviation": null,
        "sum_values": null
    }
}


**talk about how to interpret this**

In [46]:
#print("\nThe set of feature-dynamics/sub-features generate on the feature time-series",ff, sep="\n")
print(json.dumps(ff,sort_keys=True, indent=4))

{
    "F_x||length": {
        "length": null,
        "maximum": null,
        "mean": null,
        "median": null,
        "minimum": null,
        "standard_deviation": null,
        "sum_values": null,
        "variance": null
    },
    "F_x||maximum": {
        "length": null,
        "maximum": null,
        "mean": null,
        "median": null,
        "minimum": null,
        "standard_deviation": null,
        "sum_values": null,
        "variance": null
    },
    "F_x||mean": {
        "length": null,
        "maximum": null,
        "mean": null,
        "median": null,
        "minimum": null,
        "standard_deviation": null,
        "sum_values": null,
        "variance": null
    },
    "F_x||median": {
        "length": null,
        "maximum": null,
        "mean": null,
        "median": null,
        "minimum": null,
        "standard_deviation": null,
        "sum_values": null,
        "variance": null
    },
    "F_x||minimum": {
        "length": null,
     

## Select the most relevant time Series features from both of these datasets

In [144]:
## Typical feature extraction
selected_features = select_features(extracted_features,y)
display(selected_features.head())

,T_y__standard_deviation,T_y__variance,F_z__standard_deviation,F_z__variance,F_x__standard_deviation,F_x__variance,T_x__standard_deviation,T_x__variance,F_y__variance,F_y__standard_deviation,...,F_z__mean,F_z__median,F_y__maximum,F_x__minimum,T_x__minimum,F_x__maximum,T_y__minimum,T_z__maximum,T_z__minimum,F_z__maximum
1,0.471405,0.222222,1.203698,1.448889,0.249444,0.062222,0.339935,0.115556,0.115556,0.339935,...,62.533333,63.0,0.0,-1.0,-3.0,0.0,-1.0,0.0,0.0,64.0
2,2.054805,4.222222,4.333846,18.782222,0.956847,0.915556,3.422799,11.715556,4.622222,2.149935,...,62.133333,63.0,3.0,-3.0,-10.0,0.0,-5.0,0.0,-1.0,70.0
3,1.768867,3.128889,4.616877,21.315556,0.596285,0.355556,2.633122,6.933333,2.382222,1.543445,...,61.133333,61.0,2.0,-1.0,-7.0,1.0,-5.0,0.0,-1.0,68.0
4,2.669998,7.128889,3.833188,14.693333,0.952190,0.906667,3.525148,12.426667,3.982222,1.995551,...,62.200000,63.0,5.0,-2.0,-15.0,1.0,-6.0,1.0,-1.0,70.0
5,2.039608,4.160000,4.841487,23.440000,0.879394,0.773333,2.756810,7.600000,2.995556,1.730767,...,60.600000,59.0,3.0,-2.0,-12.0,2.0,-5.0,1.0,-1.0,73.0


In [145]:
selected_sub_features = select_features(extracted_sub_features,y)
display(selected_sub_features.head())

,F_z||variance__maximum,F_z||standard_deviation__maximum,T_y||variance__maximum,T_y||standard_deviation__maximum,F_z||variance__median,F_z||variance__sum_values,F_z||variance__mean,F_x||standard_deviation__maximum,F_x||variance__maximum,F_x||variance__sum_values,...,T_x||minimum__median,T_x||minimum__sum_values,T_x||minimum__mean,F_z||maximum__maximum,F_x||maximum__mean,F_x||maximum__median,F_x||maximum__sum_values,T_z||maximum__mean,T_z||maximum__median,T_z||maximum__sum_values
1,3.000000,1.732051,0.231405,0.481046,1.888430,3.776860,1.888430,0.287480,0.082645,0.082645,...,-3.0,-6.0,-3.0,64.0,-0.5,-0.5,-1.0,0.0,0.0,0.0
2,19.107438,4.371206,4.628099,2.151302,14.678719,29.357438,14.678719,1.083307,1.173554,1.361054,...,-8.0,-16.0,-8.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22.250000,4.716991,2.975207,1.724879,21.579545,43.159091,21.579545,0.655555,0.429752,0.429752,...,-7.0,-14.0,-7.0,68.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16.975207,4.120098,7.107438,2.665978,9.831353,19.662707,9.831353,1.067940,1.140496,1.390496,...,-11.5,-23.0,-11.5,70.0,0.5,0.5,1.0,0.5,0.5,1.0
5,44.750000,6.689544,4.561983,2.135880,30.019628,60.039256,30.019628,0.987525,0.975207,0.975207,...,-10.5,-21.0,-10.5,73.0,0.5,0.5,1.0,0.5,0.5,1.0


# Given this new set of subfeatures - we can decompose this into the useful features...

In [ ]:
## RUN the code with the smaller feature set
## 

# Generating new time series
potentially move this to its own notebook